In [219]:
import timeit
import pandas as pd
import numpy as np
from scipy.stats import entropy
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os

# Clean and tokenize text
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('stopwords') # download stopwords corpus
nltk.download('punkt') # download punkt tokenizer

# For linear regression
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Hannah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Hannah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## From here on preprocessing!

In [135]:
def clean_and_tokenize(text):
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('dutch'))
    tokens = [token for token in tokens if not token in stop_words]
    
    return tokens

In [136]:
df['prep_content'] = df['content'].apply(clean_and_tokenize)

AttributeError: 'float' object has no attribute 'translate'

## TRANSFORMERS

In [220]:
%cd /Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline

/Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline


In [221]:
df = pd.read_csv('Indicator-Desc_Data-High_RC.csv')

In [222]:
from transformers import AutoTokenizer, AutoModel, TFAutoModel

In [223]:
tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
#model = AutoModel.from_pretrained("GroNLP/bert-base-dutch-cased")  # PyTorch
model = TFAutoModel.from_pretrained("GroNLP/bert-base-dutch-cased")  # Tensorflow


Some layers from the model checkpoint at GroNLP/bert-base-dutch-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are newly initialized: ['bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [224]:
# Initialize an empty array to store the embeddings
embeddings = np.zeros((len(df), 768))

# Loop through the dataframe column and generate embeddings
for i, text in enumerate(df['content']):
    # Encode the text using the tokenizer
    input_ids = tokenizer.encode(str(text), add_special_tokens=True, return_tensors="tf")
    # Generate the embeddings using the model
    output = model(input_ids)
    # Extract the embeddings from the output and flatten using max-pooling
    embeddings[i] = np.max(output.last_hidden_state.numpy(), axis=1)
    
# Add the embeddings to the dataframe
for j in range(768):
    df[f'embedding_{j}'] = embeddings[:, j]


/var/folders/24/zyx78_1d5fndpjnbr8x7dd6r0000gn/T/ipykernel_7972/1907638986.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'embedding_{j}'] = embeddings[:, j]
/var/folders/24/zyx78_1d5fndpjnbr8x7dd6r0000gn/T/ipykernel_7972/1907638986.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'embedding_{j}'] = embeddings[:, j]
/var/folders/24/zyx78_1d5fndpjnbr8x7dd6r0000gn/T/ipykernel_7972/1907638986.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times

In [225]:
# Add the embeddings to the dataframe
df['embeddings'] = list(embeddings)

/var/folders/24/zyx78_1d5fndpjnbr8x7dd6r0000gn/T/ipykernel_7972/2957883369.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['embeddings'] = list(embeddings)


In [232]:
df

,name,shares,wow,cares,sad,angry,haha,reactions_count,comments,content,...,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767,embeddings
0,RTL Nieuws,1,0,11,0,0,38,67,206,"Andy en het team van de dragbar laten zich, on...",...,13.372700,1.080609,0.964037,1.063937,1.858110,1.554261,0.934504,0.737435,1.404058,"[2.1268701553344727, 1.411139726638794, 0.9980..."
1,RTL Nieuws,5,12,0,0,0,21,58,284,Alleen al de afgelopen drie dagen kwamen meer ...,...,13.318510,1.263664,0.760335,1.193509,1.659033,1.697025,1.157894,1.496184,1.604272,"[1.7587436437606812, 0.47566166520118713, 1.14..."
2,RTL Nieuws,1,0,0,0,0,0,48,4,De ploeg van bondscoach Jeroen Jacobs begint o...,...,13.383323,1.126390,1.749677,1.105100,0.999088,0.770601,1.289709,0.698110,0.850642,"[1.2422451972961426, 0.9332977533340454, 1.027..."
3,RTL Nieuws,4,17,0,0,0,52,106,16,De reddingsactie vond plaats in het Lake Distr...,...,13.290585,1.115973,0.905256,1.079573,2.682446,1.086155,0.529500,0.650274,0.569171,"[2.069547653198242, 1.149854302406311, 0.39607..."
4,RTL Nieuws,8,21,0,0,0,0,59,56,Het verkochte paar schoenen is een paar zwart-...,...,13.221995,0.850568,0.814868,1.127107,1.104677,0.801814,1.323965,0.781721,1.270635,"[1.3771954774856567, 1.095435619354248, 0.9582..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5721,De Telegraaf,3,0,0,0,0,47,103,169,Elke aardbewoner surft jaarlijks gemiddeld mee...,...,13.060578,1.298744,0.832913,1.100882,1.712117,0.040602,0.658332,1.102006,1.012051,"[2.008631706237793, 1.042741060256958, 0.94798..."
5722,De Telegraaf,4,0,0,0,0,17,60,26,De Nederlander en zijn team Red Bull hadden de...,...,13.239983,0.822294,0.695602,1.593539,1.619219,1.291816,0.724189,0.918574,1.635511,"[1.1018610000610352, 1.7308341264724731, 0.779..."
5723,De Telegraaf,2,0,0,0,23,39,71,65,„We weten weinig van de mogelijke effecten van...,...,12.788166,1.192427,0.812722,1.040017,1.249091,1.165109,0.630857,1.169717,1.338238,"[2.3504626750946045, 1.5198431015014648, 0.710..."
5724,De Telegraaf,3,0,0,0,0,60,91,135,Het rommelt binnen het CDA. De Telegraaf,...,12.757520,0.974428,0.540617,1.341013,1.987473,0.697544,0.471119,0.873903,1.283651,"[2.137939453125, 1.028130292892456, 0.32806959..."


In [227]:
df.to_csv('transformers_data-High_RC.csv', index=False)


## TF IDF

In [197]:
%cd /Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline

/Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline


In [198]:
df = pd.read_csv('Indicator-Desc_Data.csv')

In [199]:
def clean_and_tokenize(text):
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('dutch'))
    tokens = [token for token in tokens if not token in stop_words]
    
    # Join the tokens back into a single string
    text = " ".join(tokens)
    
    return tokens

In [200]:
df['content'] = df['content'].astype(str) # convert column to string data type
df['prep_content'] = df['content'].apply(clean_and_tokenize)
df['prep_content'] = df['prep_content'].astype(str) # convert column to string data type


In [203]:
## Tf-Idf (advanced variant of BoW)
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,2))

# Fit and transform the vectorizer on the text column
X = vectorizer.fit_transform(df['prep_content'])

# Convert the sparse matrix to a dense numpy array
X_array = X.toarray()

# Create a new dataframe with the tf-idf features and the original column names
tfidf_df = pd.DataFrame(X_array)

# Add the new dataframe as columns to the original dataframe
df = pd.concat([df, tfidf_df], axis=1)


In [204]:
df

,name,shares,wow,cares,sad,angry,haha,reactions_count,comments,content,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,RTL Nieuws,0,17,0,0,0,14,39,0,Medewerkers van een aquarium in Spanje aten ge...,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,RTL Nieuws,0,1,0,0,0,10,15,0,"""Er wordt wiet gerookt in het boek, gezoend, e...",...,0.0,0.0,0.0,0.0,0.0,0.297454,0.0,0.0,0.0,0.0
2,RTL Nieuws,1,0,11,0,0,38,67,206,"Andy en het team van de dragbar laten zich, on...",...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,RTL Nieuws,2,4,7,19,0,0,30,26,Onder de slachtoffers waren volgens VN-mensenr...,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,RTL Nieuws,5,12,0,0,0,21,58,284,Alleen al de afgelopen drie dagen kwamen meer ...,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,De Telegraaf,3,0,0,0,0,47,103,169,Elke aardbewoner surft jaarlijks gemiddeld mee...,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9996,De Telegraaf,4,0,0,0,0,17,60,26,De Nederlander en zijn team Red Bull hadden de...,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9997,De Telegraaf,2,0,0,0,23,39,71,65,„We weten weinig van de mogelijke effecten van...,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9998,De Telegraaf,3,0,0,0,0,60,91,135,Het rommelt binnen het CDA. De Telegraaf,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


## GLOVE

In [ ]:
import numpy as np
import fasttext as ft
from sklearn.linear_model import LinearRegression
import gensim.downloader as api
from gensim import corpora
from gensim import models
from gensim.models import KeyedVectors



In [ ]:
glove_model = gensim.models.KeyedVectors.load_word2vec_format('glove.6B.50d.txt', binary=False)

In [ ]:
def get_text_embeddings(texts, glove_model):
    """
    Convert input texts into a matrix of GloVe embeddings.
    """
    embeddings = []
    for text in texts:
        # get GloVe embedding for each word
        word_embeddings = []
        for word in text:
            # get GloVe embedding for the word
            if word in glove_model:
                word_embedding = glove_model[word]
                word_embeddings.append(word_embedding)
        # combine word embeddings into a single sentence embedding
        if word_embeddings:
            sentence_embedding = np.mean(word_embeddings, axis=0)
        else:
            sentence_embedding = np.zeros(glove_model.vector_size)
        embeddings.append(sentence_embedding)
    return np.array(embeddings)

## Linear Regression

In [ ]:
df = pd.read_csv('transformers_data-High_RC.csv') # To read in the csv file that only has posts > 40 reactions
df = pd.read_csv('transformers_data.csv') # To read in the csv file with all posts

In [228]:
# split into train and test sets
# Split the data into X and y
X = df.iloc[:, -769:-1]
#X = df['prep_content']
y = df['entropy']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [229]:
# get text embeddings for train and test sets
#X_train_embeddings = get_text_embeddings(X_train, glove_model)
#X_test_embeddings = get_text_embeddings(X_test, glove_model)

# define and train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# make predictions on the test set
y_pred = model.predict(X_test)

# evaluate model performance
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# ADD LOSS FUNCTION / INSIGHTS? maybe check overfitting with validation set. 
print(f'Mean Squared Error: {mse:.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R-squared: {r2:.2f}')

Mean Squared Error: 0.26
Mean Absolute Error: 0.41
R-squared: 0.01


## Explainer

In [ ]:
from lime.lime_text import LimeTextExplainer

# Define LimeTextExplainer
explainer = LimeTextExplainer(class_names=["target"], split_expression=" ")

# Select a random instance to explain
instance_idx = np.random.randint(len(X))
instance = X[instance_idx]
exp = explainer.explain_instance(instance, model.predict, num_features=10)

# Print the explanation
print(exp.as_list())
